<a href="https://colab.research.google.com/github/benjaminsinzore/ASP.NET-MYSQL/blob/main/Fine_Tuning_Moshi_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with Fine-Tuning Moshi 7B

This notebook shows you a simple example of how to LoRA finetune Moshi 7B. You can run this notebook in Google Colab using a A100 GPU.

<a target="_blank" href="https://colab.research.google.com/github//kyutai-labs/moshi-finetune/blob/main/tutorials/moshi_finetune.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Check out `moshi-finetune` Github repo to learn more: https://github.com/kyutai-labs/moshi-finetune/


## Installation

Clone the `moshi-finetune` repo:


In [ ]:
%cd /content/
!git clone https://github.com/kyutai-labs/moshi-finetune.git

Install all required dependencies:


In [ ]:
%pip install -e /content/moshi-finetune

## Prepare dataset


In [ ]:
from pathlib import Path
from huggingface_hub import snapshot_download
import time

Path("/content/data/daily-talk-contiguous").mkdir(parents=True, exist_ok=True)

# Download the dataset with retries and delay
local_dir = None
retries = 3  # Number of retries
delay = 5    # Delay in seconds between retries

for i in range(retries):
    try:
        local_dir = snapshot_download(
            "kyutai/DailyTalkContiguous",
            repo_type="dataset",
            local_dir="/content/data/daily-talk-contiguous",
        )
        break  # Exit loop if successful
    except Exception as e:
        if "429" in str(e):  # Check for rate limit error
            print(f"Rate limit hit. Retrying in {delay} seconds... (Attempt {i+1}/{retries})")
            time.sleep(delay)
        else:
            raise e  # Re-raise other exceptions

if local_dir is None:
    print("Failed to download dataset after multiple retries.")
else:
    print("Dataset downloaded successfully!")

## Start training


In [ ]:
# these info is needed for training
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters
import yaml

config = """
# data
data:
  train_data: '/content/data/daily-talk-contiguous/dailytalk.jsonl' # Fill
  eval_data: '' # Optionally Fill
  shuffle: true

# model
moshi_paths:
  hf_repo_id: "kyutai/moshiko-pytorch-bf16"


full_finetuning: false # Activate lora.enable if partial finetuning
lora:
  enable: true
  rank: 128
  scaling: 2.
  ft_embed: false

# training hyperparameters
first_codebook_weight_multiplier: 100.
text_padding_weight: .5


# tokens per training steps = batch_size x num_GPUs x duration_sec
# we recommend a sequence duration of 300 seconds
# If you run into memory error, you can try reduce the sequence length
duration_sec: 100
batch_size: 1
max_steps: 300

gradient_checkpointing: true # Activate checkpointing of layers

# optim
optim:
  lr: 2.e-6
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 10
eval_freq: 1
do_eval: False
ckpt_freq: 10

save_adapters: True

run_dir: "/content/test"  # Fill
"""

# save the same file locally into the example.yaml file
with open("/content/example.yaml", "w") as file:
    yaml.dump(yaml.safe_load(config), file)

In [ ]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test

In [ ]:
# start training

!cd /content/moshi-finetune && torchrun --nproc-per-node 1 -m train /content/example.yaml

## Inference

Once the model has been trained, inference can be run on the colab GPU too, and gradio can be used to tunnel the audio data from a local client to the notebook.

More details on how to set this up can be found in the [moshi readme](https://github.com/kyutai-labs/moshi?tab=readme-ov-file#python-pytorch).


In [ ]:
!pip install gradio

In [ ]:
!python -m moshi.server --gradio-tunnel --lora-weight=/content/test/checkpoints/checkpoint_000300/consolidated/lora.safetensors --config-path=/content/test/checkpoints/checkpoint_000300/consolidated/config.json